In [23]:
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def fixed_xor(b1, b2):
    a = np.frombuffer(b1, 'u1')
    b = np.frombuffer(b2, 'u1')
    return (a ^ b).tobytes()

In [6]:
K = np.random.bytes(16)
IV = np.random.bytes(16)

In [14]:
def create_encrypted_session(userdata, key=K, iv=IV):
    P0 = b"comment1=cooking%20MCs;userdata="
    P1 = b";comment2=%20like%20a%20pound%20of%20bacon"
    userdata = userdata.replace(b';', b'').replace(b'=', b'')

    padder = padding.PKCS7(128).padder()

    data = padder.update(P0 + userdata + P1) + padder.finalize()

    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).encryptor()

    return aes.update(data) + aes.finalize()

In [27]:
def decrypt_session(encrypted_session, key=K, iv=IV):
    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).decryptor()
    return aes.update(encrypted_session) + aes.finalize()

def is_admin(encrypted_session):
    plain = decrypt_session(encrypted_session)
    return b';admin=true;' in plain

In [34]:
session = create_encrypted_session(b'A' * 32)

In [58]:
blks = lambda s: [s[i:i+16] for i in range(0, len(s), 16)]

session_blks = blks(session)

n = 2

session_blks[n] = b'\x00' * 16

# a ^ b = c
# b = a ^ c

a = blks(decrypt_session(b''.join(session_blks)))[n + 1]
c = b'00;admin=true;00'
b = fixed_xor(a, c)

session_blks = blks(session)

session_blks[n] = b

blks(decrypt_session(b''.join(session_blks)))

[b'comment1=cooking',
 b'%20MCs;userdata=',
 b'\xf8|\xe9\x190\xbaQ\xbe#m\xd0Q\x84%\xe1G',
 b'00;admin=true;00',
 b';comment2=%20lik',
 b'e%20a%20pound%20',
 b'of%20bacon\x06\x06\x06\x06\x06\x06']

In [57]:
is_admin(b''.join(session_blks))

True

This works because there is no message authentication. Nothing to check that the cipher text hasn't been tempered with.